In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import shapiro, kstest, anderson, norm


In [ ]:

csv_path = "Datos_E.csv"         # Archivo
var      = "Tasa de Incidencia"  # columna a evaluar 

df = pd.read_csv(csv_path, sep=";", encoding="latin-1")
x = pd.to_numeric(df[var], errors="coerce").dropna()
n = len(x)
#conversion a varibale numerica de , a .
df["Tasa de Incidencia"] = df["Tasa de Incidencia"].str.replace(",", ".").astype(float)
x = df["Tasa de Incidencia"].dropna()
n = len(x)

In [ ]:

# Shapiro-Wilk
sw_stat, sw_p = shapiro(x if n <= 50 else x.sample(50, random_state=42))
print(f"Shapiro-Wilk: stat={sw_stat:.4f}, p={sw_p:.4f}")


In [ ]:
# Kolmogorov-Smirnov (contra N(0,1) tras estandarizar)
sigma = x.std(ddof=0)
if sigma == 0:
    print("Kolmogorov-Smirnov: σ=0 (variable constante) → no aplica.")
else:
    z = (x - x.mean()) / sigma
    ks_stat, ks_p = kstest(z, "norm")
    print(f"Kolmogorov-Smirnov: stat={ks_stat:.4f}, p={ks_p:.4f}")

In [ ]:
# Anderson-Darling
ad = anderson(x, dist='norm')
print(f"Anderson-Darling: stat={ad.statistic:.4f}  (cv 5%={ad.critical_values[2]:.4f})",
      "→ NO rechaza" if ad.statistic < ad.critical_values[2] else "→ Rechaza")

In [ ]:
# Q-Q Plot
sm.qqplot(x, line='45'); plt.title(f"Q-Q Plot: {var}"); plt.show()

# Histograma + curva normal
xs = np.linspace(x.min(), x.max(), 50)
plt.hist(x, bins=20, density=True, alpha=0.5)
plt.plot(xs, norm.pdf(xs, x.mean(), sigma), lw=2)
plt.title(f"Histograma + Normal ajustada: {var}")
plt.xlabel(var); plt.ylabel("Densidad"); plt.show()

In [26]:
# Paso 4
tamaños = [10,20,30,40,50,60,70,80,90,100]

# Opción A: filtrar tamaños mayores que n
tamaños_validos = [t for t in tamaños if t <= len(x)]

medias_muestras = {}
for n_muestra in tamaños_validos:
    muestra = x.sample(n=n_muestra, replace=True, random_state=None)
    medias_muestras[n_muestra] = muestra.mean()

print("Medias de las muestras aleatorias:")
for n_muestra, media in medias_muestras.items():
    print(f"n={n_muestra}: media={media:.2f}")



Medias de las muestras aleatorias:
n=10: media=61.42
n=20: media=61.32
n=30: media=54.87
n=40: media=54.66
n=50: media=53.22


In [32]:
# Paso 5 corregido tres muestras del mismo tamaño
muestras_20 = [x.sample(n=500, replace=True, random_state=i) for i in range(1,4)]
medias_20 = [m.mean() for m in muestras_20]

print("\nTres muestras de tamaño 20 y sus medias:")
for i, media in enumerate(medias_20, start=1):
    #dame el promedio de promedio de las tres muestras
    promedio = sum(medias_20) / len(medias_20)
    print(f"Muestra {i}: media={media:.2f}")
print(f"Promedio de las tres muestras: {promedio:.2f}")


Tres muestras de tamaño 20 y sus medias:
Muestra 1: media=52.52
Muestra 2: media=53.76
Muestra 3: media=52.95
Promedio de las tres muestras: 53.08
